# **Abstract** 
This project aims to analyze whether there is a relationship between countries' development level and their environmental and sustainability policies. The data are obtained from the World Bank for more than 200 countries between 1990 and 2023, including various socio-economic and environmental factors. 
First, different cleaning and organizing operations on the data are performed with data.table and tidyverse packages. The relationship between the selected environmental and socio-economic data is examined by cluster analysis. Then, regression and decision tree models are applied to capture both linear and non-linear relationships, providing additional insight into the socio-economic drivers of sustainability. The analysis is also supported by visuals created with the ggplot2 package to highlight country-specific trends. This study seeks to equip actionable insights by identifying key socio-economic factors that influence sustainable development globally.

# **Introduction**  
In order to cope with the global challenges posed by various environmental issues such as climate change, the relationship between environmental sustainability and socio-economic development must be understood. These dynamics need to be analyzed comprehensively, as various economic factors significantly influence environmental outcomes. This project aims to explore these relationships by utilizing a large-scale World Bank dataset covering over 30 years of data from more than 200 countries.  

The dataset consists of selected indicators reflecting environmental sustainability and socio-economic factors. Initially, cluster analysis is performed to identify natural groupings in the data, revealing patterns across countries and regions. The analysis is then deepened using regression models to quantify linear relationships and decision tree algorithms to capture complex, non-linear interactions between variables.  

The overall framework of the project includes data collection, pre-processing, clustering, modeling, and visualization. Visualization tools such as ggplot2 will facilitate the discovery of trends by providing comparative results at the country level. By integrating cluster analysis with regression and decision tree models, this project aims to provide a holistic view of the socio-economic drivers of sustainability and to reveal the relationship between countries' levels of development and environmental policies.

# **Data**
The dataset of this project includes a wide range of socio-economic and environmental indicators that are considered to reflect the dynamics of economic growth, social development, and sustainability. In order to determine the level of development, demographic metrics such as population growth and net migration are primarily selected. Socio-economic indicators include poverty, income distribution, GDP components, and access to basic resources such as electricity and health services. These data are supplemented by foreign direct investment and remittances, reflecting the effects of globalization. In order to ensure that the level of development is not only analyzed in economic terms, various social development indices are also included in the analysis. These are supported by education and human development indicators such as school enrolment rates and human capital scores, as well as data such as gender equality and female labor force participation rates that emphasize social inclusion. Environmental data covers climate and emission indicators such as greenhouse gas emissions and fossil fuel dependency, along with renewable energy use, water withdrawals, forest areas, and air pollution levels, with a focus on sustainability and stressors. 
 
During the data preparation phase, missing values (NA) were encountered in several indicators and adjustments had to be made to ensure the completeness and accuracy of the analysis. Where possible, imputation techniques, in particular the `mice` package, were applied to fill the gaps. Where missing data could not be reliably attributed or exceeded acceptable thresholds, certain indicators were either excluded or replaced with more consistently available alternatives. This process was intended to preserve the analytical depth of the dataset and minimize distortions due to missing information however may have deteriorated the reliability of the data, especially for underdeveloped countries, as too many NA values were encountered.  

With these pre-processing steps, the final dataset is ready for analysis to reveal patterns and insights into the socio-economic drivers of environmental sustainability.

# **Methods**  
The methodology includes data collection, pre-processing, clustering, regression analysis, and decision tree modeling. Following this methodology, the outcome of the analysis is intended to yield meaningful insights.

**1. Data Collection and Preprocessing**  
The content of the data and the methods applied to deal with NA values are described in detail in the Data section above.
Continuous variables were normalized to ensure consistency and comparability across countries and indicators, and the dataset was transformed using `data.table` and `tidyverse` packages to reconstruct and clean the data for analysis.

**2. Cluster Analysis**  
Cluster analysis was performed to determine the natural groupings of countries according to their environmental and socio-economic profiles. Hierarchical clustering (`hclust`) and k-means clustering were applied to the dataset. In order to determine the optimum number of clusters, the `NbClust` package, which calculates various cluster validity indices, was used.

**3. Visualization and Reporting**  
The results obtained from the cluster analysis are visualized through `ggplot2` to highlight country-based patterns and cluster differences.

**Regression Analysis**  
Since the desired result could not be obtained sufficiently with cluster analysis, it was aimed to improve the analysis by using multiple regression models to examine the linear relationships between socio-economic and environmental indicators. Regression analysis was performed using the `lm` function targeting key sustainability indicators as independent variables. Model fit and significance was evaluated through statistical tests.  

**5. Decision Tree Modelling**  
Finally, decision tree models were used to deepen the analysis by capturing non-linear interactions and complex dependencies between variables. The `rpart` package was used to build decision trees that predict environmental indicators based on socio-economic factors. The models were pruned to avoid over-fitting and the tree structures were visualized using the `visNetwork` package for clearer interpretation of decision paths.  

This methodological approach provides valuable information for researchers by combining clustering, regression, and decision tree techniques to provide a multidimensional view of socio-economic factors driving environmental sustainability.

# **Code**

In [ ]:
install.packages("mice")
install.packages("zoo")
install.packages("WDI")
install.packages("leaflet")

In [ ]:
library(tidyverse)
library(magrittr)
library(DT)
library(data.table)
library(tidyr)
library(mice)
library(zoo)
library(WDI)
library(leaflet)
library(plotly)
library(BBmisc) # for easy normalization of data
library(cluster) # for cluster analysis
library(compareGroups) # for building descriptive statistics tables
library(HDclassif) # for the dataset
library(NbClust) # for cluster validity measures
library(heatmaply) # visualize clusters with heatmap and dendrograms
library(dendextend) # enhanced dendrograms
library(circlize) # circular visualization
library(factoextra) # visualizing distances, cluster, heatmap
library(fastcluster) # faster hclust implementation
library(microbenchmark) # performance benchmarking
library(caret) # for confusion matrix
library(formattable) # for number formatting
library(pheatmap) # heatmap
library(knitr) # pretty tables
library(kableExtra) # pretty tables
library(IRdisplay) # pretty tables
library(NbClust) # cluster metrics
library(vegan) # cluster metrics
library(listviewer) # view list object
library(rpart)
library(rpart.plot)
library(psych)
library(pROC)

options(warn=-1) # for suppressing messages

In [ ]:
file_path <- "https://raw.githubusercontent.com/s-gkce/AD48G_finalProjectData/refs/heads/main/wb_dt.csv" 
data <- fread(file_path)

In [ ]:
print(names(data))

In [ ]:
year_columns <- names(data)[grep("\\[YR", names(data))]

## Reshaping the data for easier analysis

In [ ]:
melted_data <- melt(data, 
                    id.vars = c("Country Name", "Country Code", "Series Name", "Series Code"),
                    measure.vars = year_columns,
                    variable.name = "Year",
                    value.name = "Value")

In [ ]:
head(melted_data)

In [ ]:
melted_data[, Year := substr(Year, 1, 4)]

In [ ]:
melted_data[Value %in% c("..", "-", "", "n/a", "No data"), Value := NA]

In [ ]:
melted_data[, Value := as.numeric(Value)]

In [ ]:
melted_data <- melted_data[!is.na(`Country Name`) & `Country Name` != "", ]

In [ ]:
head(melted_data)

In [ ]:
unique_series_names <- unique(melted_data$`Series Name`)
num_unique_series_names <- length(unique_series_names)

print(num_unique_series_names)
print(unique_series_names)

In [ ]:
setnames(melted_data, old = c("Series Name", "Country Name", "Country Code", "Series Code"),
         new = c("seriesName", "countryName", "countryCode", "seriesCode"))

head(melted_data)

In [ ]:
grouped_data <- setorder(melted_data, countryName) 

In [ ]:
head(grouped_data)

In [ ]:
grouped_data[, seriesName := paste0(seriesName, " [", seriesCode, "]")]

grouped_data[, seriesCode := NULL]

In [ ]:
pivoted_data <- grouped_data %>%
  pivot_wider(names_from = seriesName,
              values_from = Value,
              values_fn = list(Value = function(x) if (length(x) > 1) mean(x, na.rm = TRUE) else x)) %>%
  as.data.table()

In [ ]:
head(pivoted_data)

In [ ]:
data <- pivoted_data %>% 
  rename(
  "country" = "countryName",
  "code" = "countryCode",
  "year" = "Year",
  "Pop_Growth" = "Population growth (annual %) [SP.POP.GROW]",
  "Poverty_Headcount" = "Poverty headcount ratio at national poverty lines (% of population) [SI.POV.NAHC]",
  "Income_Low20" = "Income share held by lowest 20% [SI.DST.FRST.20]",
  "Life_Expectancy" = "Life expectancy at birth, total (years) [SP.DYN.LE00.IN]",
  "Water_Productivity" = "Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal) [ER.GDP.FWTL.M3.KD]",
  "Energy_Use" = "Energy use (kg of oil equivalent per capita) [EG.USE.PCAP.KG.OE]",
  "GDP_Growth" = "GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]",
  "Agriculture_GDP" = "Agriculture, forestry, and fishing, value added (% of GDP) [NV.AGR.TOTL.ZS]",
  "Industry_GDP" = "Industry (including construction), value added (% of GDP) [NV.IND.TOTL.ZS]",
  "Tech_Exports" = "High-technology exports (% of manufactured exports) [TX.VAL.TECH.MF.ZS]",
  "Net_Migration" = "Net migration [SM.POP.NETM]",
  "Remittances" = "Personal remittances, paid (current US$) [BM.TRF.PWKR.CD.DT]",
  "GDP_Per_Capita" = "GDP per capita (current US$) [NY.GDP.PCAP.CD]",
  "FDI" = "Foreign direct investment, net (BoP, current US$) [BN.KLT.DINV.CD]",
  "Inflation" = "Inflation, consumer prices (annual %) [FP.CPI.TOTL.ZG]",
  "Electricity_Access" = "Access to electricity (% of population) [EG.ELC.ACCS.ZS]",
  "Freshwater_Withdrawals" = "Annual freshwater withdrawals, total (% of internal resources) [ER.H2O.FWTL.ZS]",
  "Water_Stress" = "Level of water stress: freshwater withdrawal as a proportion of available freshwater resources [ER.H2O.FWST.ZS]",
  "Protected_Areas" = "Terrestrial and marine protected areas (% of total territorial area) [ER.PTD.TOTL.ZS]",
  "Unemployment" = "Unemployment, total (% of total labor force) (national estimate) [SL.UEM.TOTL.NE.ZS]",
  "Gender_Equality" = "CPIA gender equality rating (1=low to 6=high) [IQ.CPA.GNDR.XQ]",
  "Female_Labor_Force" = "Labor force, female (% of total labor force) [SL.TLF.TOTL.FE.ZS]",
  "Health_Expenditure" = "Current health expenditure (% of GDP) [SH.XPD.CHEX.GD.ZS]",
  "Renewable_Energy" = "Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS]",
  "Renewable_Electricity" = "Renewable electricity output (% of total electricity output) [EG.ELC.RNEW.ZS]",
  "Forest_Area" = "Forest area (% of land area) [AG.LND.FRST.ZS]",
  "Agricultural_Land" = "Agricultural land (% of land area) [AG.LND.AGRI.ZS]",
  "Alt_Nuclear_Energy" = "Alternative and nuclear energy (% of total energy use) [EG.USE.COMM.CL.ZS]",
  "Energy_Depletion" = "Adjusted savings: energy depletion (% of GNI) [NY.ADJ.DNGY.GN.ZS]",
  "PM25_Exposure" = "PM2.5 air pollution, mean annual exposure (micrograms per cubic meter) [EN.ATM.PM25.MC.M3]",
  "Resource_Depletion" = "Adjusted savings: natural resources depletion (% of GNI) [NY.ADJ.DRES.GN.ZS]",
  "Forest_Depletion" = "Adjusted savings: net forest depletion (% of GNI) [NY.ADJ.DFOR.GN.ZS]",
  "Gini_Index" = "Gini index [SI.POV.GINI]",
  "Business_Environment" = "CPIA business regulatory environment rating (1=low to 6=high) [IQ.CPA.BREG.XQ]",
  "Social_Inclusion" = "CPIA policies for social inclusion/equity cluster average (1=low to 6=high) [IQ.CPA.SOCI.XQ]",
  "Transparency" = "CPIA transparency, accountability, and corruption in the public sector rating (1=low to 6=high) [IQ.CPA.TRAN.XQ]",
  "Human_Resources" = "CPIA building human resources rating (1=low to 6=high) [IQ.CPA.HRES.XQ]",
  "GHG_Ex_LULUCF" = "Total greenhouse gas emissions excluding LULUCF (% change from 1990) [EN.GHG.TOT.ZG.AR5]",
  "GHG_Incl_LULUCF" = "Total greenhouse gas emissions including LULUCF (Mt CO2e) [EN.GHG.ALL.LU.MT.CE.AR5]",
  "CH4_Emissions" = "Methane (CH4) emissions (total) excluding LULUCF (% change from 1990) [EN.GHG.CH4.ZG.AR5]",
  "CO2_Emissions" = "Carbon dioxide (CO2) emissions (total) excluding LULUCF (% change from 1990) [EN.GHG.CO2.ZG.AR5]",
  "Fossil_Fuel_Use" = "Fossil fuel energy consumption (% of total) [EG.USE.COMM.FO.ZS]",
  "Female_Male_Labor" = "Ratio of female to male labor force participation rate (%) (modeled ILO estimate) [SL.TLF.CACT.FM.ZS]",
  "Primary_Enrollment" = "School enrollment, primary, private (% of total primary) [SE.PRM.PRIV.ZS]",
  "Human_Capital" = "Human capital index (HCI) (scale 0-1) [HD.HCI.OVRL]"
)

In [ ]:
str(data)

In [ ]:
data %>%
  arrange(country, year)%>% datatable(
  filter = "top",
  options = list(pageLength = 20)
)

In [ ]:
colSums(is.na(data))

In [ ]:
na_summary <- data %>%
  group_by(country) %>%
  summarise_all(~ sum(is.na(.))) %>%
datatable(
  filter = "top",
  options = list(pageLength = 20)
)

na_summary

In [ ]:
na_summary <- colSums(is.na(data))

all_na_columns <- names(na_summary[na_summary == nrow(data)])

na_percentage <- na_summary / nrow(data) * 100

print(na_summary)
print(paste("NA filled Columns:", paste(all_na_columns, collapse = ", ")))
print("Missing Data %:")
print(na_percentage)

In [ ]:
country_na_summary <- data[, lapply(.SD, function(x) mean(is.na(x)) > 0.8), 
                                   by = .(country, code)]

country_na_count <- country_na_summary[, .(High_NA_Count = rowSums(.SD)), 
                                       by = .(country, code), 
                                       .SDcols = !c("country", "code")]
                                    
country_na_count%>%
datatable(
  filter = "top",
  options = list(pageLength = 20)
)

Since there are too many NA values, it was thought that removing countries with too many NA values would give a more accurate result for the analysis.

In [ ]:
high_na_countries <- country_na_count[High_NA_Count > 20, country]

data2 <- data[!country %in% high_na_countries]

In [ ]:
colSums(is.na(data2))
nrow(data2)

Still, not enough to make an efficient analysis. Some other arrangements should be made.

In [ ]:
na_summary2 <- colSums(is.na(data2))

all_na_columns2 <- names(na_summary2[na_summary2 == nrow(data2)])

na_percentage2 <- na_summary2 / nrow(data2) * 100

print(paste("NA filled Columns:", paste(all_na_columns2, collapse = ", ")))
print("Missing Data %:")
print(na_percentage2)

**There are still too many NA values, but the analysis needs to be continued. Therefore, various methods are used to fill the NA values, with efforts made not to distort the original data too much.** 

## Dealing with NA

Different methods were applied to handle missing values (NA). For columns with less than 40% missing data (critical_cols), forward and backward filling were used, as this method efficiently imputes small gaps by carrying over the nearest existing value, preserving the overall distribution and minimizing data distortion. However, for columns with 40% to 80% missing data (moderate_cols), the MICE (Multiple Imputation by Chained Equations) method was used. This approach, being more sophisticated, is better suited for handling larger gaps, as it estimates missing values by modeling the relationship between variables. Since the focus of this course is more on the data analysis process, this approach was followed to continue the analysis, even though it may harm the reliability of the data.

In [ ]:
setDT(data2)
na_percent <- sapply(data2, function(x) mean(is.na(x)) * 100)

# group according to NA values
critical_cols <- names(na_percent[na_percent < 40])  
moderate_cols <- names(na_percent[na_percent >= 40 & na_percent <= 80]) 
high_na_cols <- names(na_percent[na_percent > 80]) 

num_cols <- names(data2)[sapply(data, is.numeric)]
char_cols <- names(data2)[sapply(data, is.character)]

# Forward-backward fill 
for (col in critical_cols) {
  data2[, (col) := na.locf(get(col), na.rm = FALSE)]
  data2[, (col) := na.locf(get(col), fromLast = TRUE, na.rm = FALSE)]
}

# MICE for moderate cols
mice_subset <- data2[, c("country", "code", moderate_cols), with = FALSE]

mice_imputed <- mice(mice_subset, m = 5, method = 'pmm', maxit = 10)

mice_completed <- as.data.table(complete(mice_imputed))

data2[, (moderate_cols) := mice_completed[, moderate_cols, with = FALSE]]

summary(data)

In [ ]:
colSums(is.na(data2))

## Data Analysis

In [ ]:
str(data2)

In [ ]:
data2 %>% 
  keep(is.numeric) %>% 
  summarise(across(everything(), list(
    n = ~sum(!is.na(.)),
    mean = ~round(mean(., na.rm = TRUE), 2),
    sd = ~round(sd(., na.rm = TRUE), 2),
    median = ~round(median(., na.rm = TRUE), 2),
    min = ~round(min(., na.rm = TRUE), 2),
    max = ~round(max(., na.rm = TRUE), 2)
  ), .names = "{.col}_{.fn}"))

## Cluster

This code clusters countries based on development indicators. First, key indicators are selected. Rows with missing data are removed to ensure accuracy. The data (excluding country and code) is scaled for equal contribution to clustering. The elbow method helps determine the optimal number of clusters by plotting WSS values. K-means clustering is performed with 3 clusters, and the results are saved in a table linking each country to its cluster. The cluster labels are then merged with the main dataset for further analysis or visualization.

In [ ]:
development_indicators <- data2[, .(
  country, code, GDP_Per_Capita, GDP_Growth, FDI, Industry_GDP, Agriculture_GDP,
  Net_Migration, Inflation, Life_Expectancy, Electricity_Access,
  Primary_Enrollment, Female_Labor_Force, Gini_Index,
  Human_Capital, Gender_Equality, Transparency, Social_Inclusion
)]

development_indicators <- na.omit(development_indicators)

scaled_data <- scale(development_indicators[, -c("country", "code")])

wss <- function(k) kmeans(scaled_data, k, nstart = 10)$tot.withinss
k.values <- 1:10
wss_values <- sapply(k.values, wss)
plot(k.values, wss_values, type = "b", pch = 19)

set.seed(123)
clusters <- kmeans(scaled_data, centers = 3) 

clustered_data <- data.table(
  country = development_indicators$country,
  Cluster = clusters$cluster
)

data3 <- copy(data2)

data3[clustered_data, on = "country", Cluster := i.Cluster]

Above graph shows the elbow method, suggesting the optimal number of clusters is around 3 or 4, where the WSS starts to level off.

In [ ]:
data3[, .N, by = Cluster]

In [ ]:
data3[is.na(Cluster), unique(country)]

Since these NA values are belong to just some island countries, they will be ignored and evaluated in another cluster.

In [ ]:
data3[is.na(Cluster), Cluster := 0]

In [ ]:
data3[, .N, by = Cluster]

In [ ]:
cluster_summary <- data3[, lapply(.SD, mean, na.rm = TRUE), by = Cluster,
                         .SDcols = c("GDP_Per_Capita", "GDP_Growth", "FDI", "Industry_GDP", "Agriculture_GDP",
  "Net_Migration", "Inflation", "Life_Expectancy", "Electricity_Access",
  "Primary_Enrollment", "Female_Labor_Force", "Female_Male_Labor", "Gini_Index",
  "Human_Capital", "Gender_Equality", "Transparency", "Social_Inclusion")]
print(cluster_summary)

As a result of the cluster analysis, four different groups were identified. Cluster 1 most likely represents developed countries with high GDP per capita (20,691 USD), full electricity access (98 percent), long life expectancy (75.6 years), and net migration (+43,549). These countries have a high female labor force participation rate (44.95%) and moderate transparency (2.99). Cluster 2 represents less developed countries with indicators such as low GDP per capita (USD 1,382), limited access to electricity (45 percent), short life expectancy (57.1 years), high inflation (38 percent), and net emigration (-7,655). In this cluster, the female labor force participation rate remains moderate (45.9%). Cluster 3 represents developing countries with medium GDP (7,001 USD), good access to electricity (88 percent), relatively long life expectancy (70.6 years), and high emigration (-38,846). Moreover, in this cluster, foreign investments are negative and capital outflows are observed. Cluster 0 is similar to Cluster 3 in terms of GDP per capita (USD 8,416) and life expectancy (68.7 years), but is slightly better off and represents upper-middle-income developing countries. In these countries, electricity access (82.6 percent) and inflation (13 percent) are moderate, and primary school enrolment is high (19.86). In general, the socio-economic development levels of countries can be grouped as follows:
- **Cluster 1 →** Developed Countries, 
- **Cluster 2 →** Less Developed Countries, 
- **Cluster 3 →** Developing Countries, 
- **Cluster 0 →** Upper-Middle Income Countries.

### Evaluating these clusters in a world map
In order to use a world map, the latitude and longitude values of each country are needed. These values are taken from the WDI dataset, but some manual editing is required because the names of certain countries do not match.

In [ ]:
wdi_country_info <- as.data.table(WDI::WDI(country = "all", indicator = "NY.GDP.PCAP.CD", extra = TRUE))

wdi_country_info <- wdi_country_info[!is.na(region), .(country, region, income, latitude, longitude)]

wdi_country_info[country == "Vietnam", country := "Viet Nam"]
wdi_country_info[country == "Czech Republic", country := "Czechia"]

unique_country_data3 <- unique(data3[, .(country)])

wdi_country_info <- wdi_country_info[country %in% unique_country_data3$country]

data4 <- data3[wdi_country_info, on = "country", `:=`(
  region = i.region,
  income = i.income,
  latitude = i.latitude,
  longitude = i.longitude
)]

In [ ]:
head(data4)

For some further analysis, columns are edited.

In [ ]:
factor_cols <- c("country", "code", "year", "region", "income")

numeric_cols <- c("latitude", "longitude")

data4[, (factor_cols) := lapply(.SD, as.factor), .SDcols = factor_cols]

data4[, (numeric_cols) := lapply(.SD, as.numeric), .SDcols = numeric_cols]

str(data4)

### Cluster Values in the Map

In [ ]:
plot_geo(data4, locationmode = "ISO-3") %>%
  add_trace(
    z = ~Cluster, text = ~country,
    locations = ~code, color = ~Cluster
  ) %>%
  colorbar(title = "Cluster") %>%
  layout(
    geo = list(showframe = FALSE)
  )

The map visually represents the clustering of countries based on development indicators. As it is given above, cluster analysis:  

- **Cluster 1 (Blue)** – Represents developed countries with high GDP per capita, long life expectancy, and full electricity access. These countries attract migration.  
- **Cluster 2 (Green)** – Represents less developed countries with low GDP, poor electricity access, and shorter life expectancy. These countries often face high inflation and emigration.  
- **Cluster 3 (Yellow)** – Indicates developing countries with moderate GDP, good electricity access, and average life expectancy. However, they experience negative net migration.  
- **Cluster 0 (Purple)** – Suggests upper-middle-income countries with improving indicators but facing economic challenges.  

However, some inconsistencies appear when comparing the results to the expected patterns. For example, while Cluster 1 generally aligns with developed countries, a few regions classified under it might have been expected to fall into lower development categories. Similarly, some countries in Cluster 3 may not fully reflect the characteristics of developing countries.

These discrepancies could stem from data gaps, particularly for less developed countries, or from the methods used to handle missing values (such as forward-backward filling or MICE). The imputation process may have introduced bias, affecting the clustering outcome. This highlights the limitations of relying on incomplete data for classification.

## Cluster for Environmental Analysis

It is believed that accurate results may not have been produced by the initial clustering model because the number of clusters was fixed at \( k = 3 \) without being fully aligned with the outcome of the elbow method. To address this, the elbow method was adjusted to identify the optimal cluster size (\( k = 4 \)), ensuring that the natural structure of the data was better reflected in the clustering process. Additionally, it was observed that too many variables were included in the initial cluster model, potentially leading to redundancy and imbalance. To mitigate this, the number of variables was reduced by focusing on key economic indicators. These adjustments were aimed at preventing over- or under-clustering, resulting in groupings that were more balanced and meaningful.

In [ ]:
# Extracting key economic and development indicators from the dataset for development cluster
development_indicators <- data2[, .(
  country, code, GDP_Per_Capita, GDP_Growth, FDI, Industry_GDP, 
  Agriculture_GDP, Net_Migration, Inflation, Life_Expectancy,
  Electricity_Access, Primary_Enrollment, Female_Labor_Force,
  Gini_Index, Human_Capital
)]

development_indicators <- na.omit(development_indicators)

scaled_dev_data <- scale(development_indicators[, -c("country", "code")])

wss_dev <- function(k) {
  kmeans(scaled_dev_data, k, nstart = 10)$tot.withinss 
}
k.values <- 1:10 
wss_values_dev <- sapply(k.values, wss_dev) 

plot(k.values, wss_values_dev, type = "b", pch = 19,
     xlab = "Number of Clusters (K)",
     ylab = "Total Within-Cluster Sum of Squares (WSS)")

set.seed(123)
dev_clusters <- kmeans(scaled_dev_data, centers = 4)

dev_clustered_data <- data.table(
  country = development_indicators$country,
  Dev_Cluster = dev_clusters$cluster
)

#Environmental Cluster
environmental_indicators <- data2[, .(
  country, code, Renewable_Energy, Fossil_Fuel_Use, CO2_Emissions,
  CH4_Emissions, GHG_Incl_LULUCF, Forest_Area, Water_Stress,
  Alt_Nuclear_Energy, PM25_Exposure, Resource_Depletion
)]

environmental_indicators <- na.omit(environmental_indicators)

scaled_env_data <- scale(environmental_indicators[, -c("country", "code")])

wss_env <- function(k) {
  kmeans(scaled_env_data, k, nstart = 10)$tot.withinss
}
wss_values_env <- sapply(k.values, wss_env)

plot(k.values, wss_values_env, type = "b", pch = 19,
     xlab = "Number of Clusters (K)",
     ylab = "Total Within-Cluster Sum of Squares (WSS)")

set.seed(123)
env_clusters <- kmeans(scaled_env_data, centers = 5)

env_clustered_data <- data.table(
  country = environmental_indicators$country,
  Env_Cluster = env_clusters$cluster
)

data5 <- copy(data2)

data5[dev_clustered_data, on = "country", Dev_Cluster := i.Dev_Cluster]

data5[env_clustered_data, on = "country", Env_Cluster := i.Env_Cluster]

cross_tab <- table(data5$Dev_Cluster, data5$Env_Cluster)
print(cross_tab)

correlation_analysis <- cor(data5[, .(
  GDP_Per_Capita, Renewable_Energy, Fossil_Fuel_Use,
  CO2_Emissions, CH4_Emissions, Forest_Area, Water_Stress
)], use = "complete.obs")
print(correlation_analysis)

In this cluster analysis, separate clusters were created for environmental and development indicators to demonstrate the relationship between the two contexts. First, the appropriate number of clusters is determined by using the elbow method and then the relationship between the clusters is tried to be observed with the correlation matrix. The correlation matrix shows that cluster analysis reveals that highly developed countries (Dev_Cluster 1) are concentrated in environmentally moderate or industrialized clusters, while developing countries (Dev_Cluster 2 and 3) show different environmental performance. Lower development clusters (Dev_Cluster 3) are associated with higher environmental stress. The correlation analysis highlights weak relationships between GDP per capita and fossil fuel use (0.15) or renewable energy (-0.33), pointing to various energy strategies. Moderate overlaps in CO2 and CH4 emissions (0.28) reflect industrial and agricultural impacts.

In [ ]:
data5[, .N, by = Dev_Cluster]
data5[, .N, by = Env_Cluster]

In [ ]:
data5[is.na(Dev_Cluster), unique(country)]

Since these NA values are belong to just some island countries, I will ignore them again and evaluate them in another cluster.

In [ ]:
data5[is.na(Dev_Cluster), Dev_Cluster := 0]

In [ ]:
data5[, .N, by = Dev_Cluster]

In [ ]:
# Cluster Summaries
env_cluster_summary <- data5[, lapply(.SD, mean, na.rm = TRUE), by = Env_Cluster,
                             .SDcols = c("Renewable_Energy", "Fossil_Fuel_Use", 
                                         "CO2_Emissions", "CH4_Emissions", 
                                         "GHG_Incl_LULUCF", "Forest_Area", 
                                         "Water_Stress", "PM25_Exposure")]
print(env_cluster_summary)

dev_cluster_summary <- data5[, lapply(.SD, mean, na.rm = TRUE), by = Dev_Cluster,
                             .SDcols = c("GDP_Per_Capita", "GDP_Growth", 
                                         "Life_Expectancy", "Electricity_Access",
                                         "Primary_Enrollment", "Female_Labor_Force",
                                         "Gini_Index")]
print(dev_cluster_summary)


With this cluster analysis, different patterns between environmental sustainability and socioeconomic development can be observed across clusters. Developed countries (Cluster 1) have the highest renewable energy use (61.28 percent), GDP per capita, and life expectancy (78.02) but are still significantly dependent on fossil fuels (47.35 percent) and face moderate emissions. In contrast, less developed countries (Cluster 5) exhibit low renewable energy use (16.68 percent), high fossil fuel dependence (76.41 percent), severe water stress (587.34), and limited GDP per capita ($1,854.16). Transition clusters (3 and 4) show mixed features, negating moderate renewable energy use with fossil fuel dependency and changing environmental pressures. These findings suggest that developed countries are not in a good position in terms of their sustainability policies. It can be argued that special policies are needed to increase sustainability and address inequalities between development stages. 

In [ ]:
plot_geo(data5, locationmode = "ISO-3") %>%
  add_trace(
    z = ~Dev_Cluster, text = ~country,
    locations = ~code, color = ~Dev_Cluster
  ) %>%
  colorbar(title = "Development Cluster") %>%
  layout(
    geo = list(showframe = FALSE)
  )

plot_geo(data5, locationmode = "ISO-3") %>%
  add_trace(
    z = ~Env_Cluster, text = ~country,
    locations = ~code, color = ~Env_Cluster
  ) %>%
  colorbar(title = "Environmental Cluster") %>%
  layout(
    geo = list(showframe = FALSE)
  )


When cluster analyses are placed on the world map for observation, some unexpected situations are encountered. The new cluster on the level of development of countries provides more consistent information, but the result of the environmental cluster is unexpected. With the information obtained from the cluster summary, it would be correct to rank the environmental clusters from best to worst as follows: Cluster 1 is the most sustainable with the highest renewable energy use (61.28%), the lowest fossil fuel dependency (47.35%), minimal water stress and the largest forest area. Cluster 4 and 2 follow with moderate renewable energy use and relatively balanced environmental indicators. Cluster 3 faces higher emissions and moderate water stress despite moderate renewable energy use. Finally, Cluster 5 reflects severe challenges, ranking worst with the lowest renewable energy use (16.68 percent), highest fossil fuel dependency (76.41 percent), extreme water stress, and minimal forest area. In this respect, it was surprising to find several African and Asian countries in the best environmental cluster. This may be due to the lack of data for these regions and the methods used to cope with the lack of data. A more expected result is that most of the countries defined as developed are observed in clusters 2 and 4.

## Regression

In [ ]:
reg_model <- lm(GDP_Per_Capita ~ GHG_Incl_LULUCF + Water_Stress + PM25_Exposure + Protected_Areas +
                Renewable_Energy + Fossil_Fuel_Use + Water_Productivity + Energy_Use + Freshwater_Withdrawals + 
                Energy_Depletion + Resource_Depletion + Forest_Area, data = data5)

summary(reg_model)

In [ ]:
geometric_mean <- function(x) {
  exp(mean(log(x), na.rm = TRUE))
}

development_indicators <- data2[, .(
  country, code, GDP_Per_Capita, GDP_Growth, FDI, Industry_GDP, 
  Agriculture_GDP, Net_Migration, Inflation, Life_Expectancy,
  Electricity_Access, Primary_Enrollment, Female_Labor_Force,
  Gini_Index, Human_Capital
)]

development_indicators[, target_variable := apply(.SD, 1, geometric_mean), 
                       .SDcols = c("GDP_Per_Capita", "GDP_Growth", "FDI", "Industry_GDP", 
                                   "Agriculture_GDP", "Net_Migration", "Inflation", "Life_Expectancy",
                                   "Electricity_Access", "Primary_Enrollment", "Female_Labor_Force",
                                   "Gini_Index", "Human_Capital")]

data6 <- merge(data5, development_indicators[, .(country, target_variable)], by = "country", allow.cartesian = TRUE)

data6[, target_category := ifelse(target_variable <= median(target_variable, na.rm = TRUE), "Actual", "Predict")]

data6$target_category <- factor(data6$target_category, levels = c("Actual", "Predict"))

In [ ]:
regression_model <- lm(target_variable ~ GHG_Incl_LULUCF + Water_Stress + PM25_Exposure + 
              Protected_Areas + Renewable_Energy + Fossil_Fuel_Use + Water_Productivity + 
              Energy_Use + Freshwater_Withdrawals + Energy_Depletion + 
              Resource_Depletion + Forest_Area, data = data6)

summary(regression_model)

In the regression analysis, to compare the development level with environmental factors, a regression was initially conducted between a straightforward variable, such as per capita GDP, representing development level, and the identified environmental factors. Although the resulting R² value was not unsatisfactory, a different approach was adopted, as it would not be reasonable to define development solely based on GDP per capita. Therefore, adjustments were made to the data to enable a more comprehensive regression analysis. Specifically, to create a suitable target variable from the columns in the previously defined *development_indicators*, the geometric means of these columns were calculated to ensure compatibility for regression. Subsequently, the identified sustainability indicators were analyzed using the created target variables to explore how each value influenced the target variable and to what extent.

Regression analysis returned a number of significant relations of the selected environmental indicators with the target variable: GHG_Incl_LULUCF, water stress, and protected areas were strongly positively related to the target variable at \(p < 0.001\), hence highly influential. On the other hand, PM2.5 exposure, renewable energy use, and freshwater withdrawals were significantly negatively related to the target variable at \(p < 0.001\). Interestingly, there was no significant relationship from the use of fossil fuel (p = 0.783), showing the complexity in its role. Although these findings gave a fair idea about the relationship between environmental factors and development, the generally low adjusted R² value of 0.0962 revealed the weakness of the model in explaining the phenomenon and served as an indication that the models require more variable inclusions or complexity to capture the full dynamics. This version includes all the important regression results in a context that makes sense.

## Decision Tree Analysis

In [ ]:
tree_model <- rpart(target_category ~ GHG_Incl_LULUCF + Water_Stress + PM25_Exposure + 
                                 Protected_Areas + Renewable_Energy + Fossil_Fuel_Use + Water_Productivity + 
                                 Energy_Use + Freshwater_Withdrawals + Energy_Depletion + 
                                 Resource_Depletion + Forest_Area, data = data6, method = "class")

visNetwork::visTree(tree_model, main = "Decision Tree")

In [ ]:
pred <- predict(tree_model, data6, type = "class")

actual <- data6$target_category

pred <- factor(pred, levels = levels(actual))

conf_matrix <- confusionMatrix(pred, actual)
print(conf_matrix)

pred_prob <- predict(tree_model, data6, type = "prob")[,2]
roc_curve <- roc(actual, pred_prob, levels = levels(actual))
plot(roc_curve)
auc_value <- auc(roc_curve)
print(paste("AUC: ", auc_value))

### Pruned Tree

In [ ]:
printcp(tree_model)
best_cp <- tree_model$cptable[which.min(tree_model$cptable[,"xerror"]),"CP"]

pruned_tree <- prune(tree_model, cp = best_cp)

visNetwork::visTree(pruned_tree, main = "Pruned Decision Tree")

In [ ]:
pred_pruned <- predict(pruned_tree, data6, type = "class")

actual <- data6$target_category

pred_pruned <- factor(pred_pruned, levels = levels(actual))

conf_matrix_pruned <- confusionMatrix(pred_pruned, actual)
print(conf_matrix_pruned)

pred_prob_pruned <- predict(pruned_tree, data6, type = "prob")[,2]
roc_curve_pruned <- roc(actual, pred_prob_pruned, levels = levels(actual))
plot(roc_curve_pruned)
auc_value_pruned <- auc(roc_curve_pruned)
print(paste("Pruned AUC: ", auc_value_pruned))

Finally, to enhance the analysis further, a decision tree was employed to explore the nonlinear relationships between the target_variable created in the previous step and the identified environmental sustainability factors. The accuracy of the decision tree's results was evaluated using a confusion matrix, and the AUC value was calculated and visualized with the ROC curve. As the initial accuracy values were slightly higher than expected, the analysis was pruned following a "too good to be true" approach, and the calculations were repeated to ensure reliability.

The confusion matrix for the initial decision tree, which provided an accuracy of 67.84%, with a sensitivity of 73.94% and a specificity of 61.74%. The estimated AUC value for the unpruned model is 0.695, shown on the ROC curve. As these initial accuracy values looked somewhat higher than intuitively felt, the model was pruned to avoid overfitting, using the complexity parameter with the lowest cross-validated error. This is reflected in the pruned decision tree that made its predictions using fewer variables, mainly **Energy Depletion, Energy Use, PM2.5 Exposure, Renewable Energy, Water Productivity**, and **Water Stress**.

After pruning, the model's accuracy remained at 67.84%, while sensitivity and specificity showed relatively similar values. Surprisingly, the AUC for the pruned tree showed a slight increase in value to 0.6955, reflecting better reliability and a closer generalization capability of the model. These results are indicative that, though pruning simplified the model, the predictive power of the model was preserved and potential overfitting was avoided. Based on the above presented decision tree analysis, combined with the regression and clustering models, the study puts into focus complex and nonlinear relationships of socio-economic factors with environmental sustainability for actionable insights on drivers of sustainable development.

# Conclusion
The main objective of this project was to analyze the relationship between countries’ development levels and environmental sustainability policies using a comprehensive dataset covering more than 200 countries spanning over 30 years. The study used clustering, regression and decision tree techniques to reveal patterns, quantify relationships and examine the main socio-economic drivers of sustainability. While some insights were obtained, the analysis faced certain difficulties and limitations.

The regression analysis did not yield a very poor R^2 result when development was reduced to a single value, but when a complex development variable was created by taking the geometric mean, it yielded very low R² values. The low explanatory power of the analysis indicated that the linear relationships between the selected socio-economic and environmental indicators were weaker than expected. Similarly, the cluster analysis provided valuable groupings of countries according to their development and environmental characteristics, but when visualized on global maps, no direct overlap was observed between development and environmental clusters. This discrepancy highlights the complexity of the relationship and may indicate that development and environmental performance are affected by additional, unobserved factors or that our methods of dealing with the NA values ​​at the beginning of the analysis produced a worse result than expected.

Despite these limitations, the analysis achieved some success by identifying separate clusters and revealing nonlinear relationships through decision tree modeling. These approaches revealed important trends, such as the association of lower development clusters with higher environmental stress and the mixed performance of transition economies.

Although the project did not fully achieve its goal of robustly explaining the relationship between development and environmental sustainability, it did provide valuable insights and highlight the significant resulting challenges in analyzing these dynamics. The findings highlight the need for more comprehensive data, refined models, and additional contextual factors to better understand and address the complex interactions between socioeconomic development and environmental sustainability.